<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/TG_Bot_%2B_GPT3_5_Turbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# Устанавливаем необходимые библиотеки
!pip install aiogram
!pip install nest_asyncio
!pip install mwclient
!pip install mwparserfromhell
!pip install openai
!pip install tiktoken

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
# Импортируем необходимые библиотеки
import asyncio
import logging
from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command
import warnings
warnings.filterwarnings('ignore')
import nest_asyncio
nest_asyncio.apply()
import time
import mwclient
import mwparserfromhell
import openai
import pandas as pd
import re
import tiktoken
from openai import OpenAI
import os
import getpass
from scipy import spatial

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
# Подготовка данных из Википедии про MIT
CATEGORY_TITLE = "Category:Massachusetts Institute of Technology"
WIKI_SITE = "en.wikipedia.org"

# Соберем заголовки всех статей
def titles_from_category(
    category: mwclient.listing.Category, # Задаем типизированный параметр категории статей
    max_depth: int # Определяем глубину вложения статей
) -> set[str]:
    """Возвращает набор заголовков страниц в данной категории Википедии и ее подкатегориях."""
    titles = set() # Используем множество для хранения заголовков статей
    for cm in category.members(): # Перебираем вложенные объекты категории
        if type(cm) == mwclient.page.Page: # Если объект является страницей
            titles.add(cm.name) # в хранилище заголовков добавляем имя страницы
        elif isinstance(cm, mwclient.listing.Category) and max_depth > 0: # Если объект является категорией и глубина вложения не достигла максимальной
            deeper_titles = titles_from_category(cm, max_depth=max_depth - 1) # вызываем рекурсивно функцию для подкатегории
            titles.update(deeper_titles) # добавление в множество элементов из другого множества
    return titles

# Инициализация объекта MediaWiki
# WIKI_SITE ссылается на англоязычную часть Википедии
site = mwclient.Site(WIKI_SITE)

# Загрузка раздела заданной категории
category_page = site.pages[CATEGORY_TITLE]
# Получение множества всех заголовков категории с вложенностью на один уровень
titles = titles_from_category(category_page, max_depth=1)

print(f"Создано {len(titles)} заголовков статей в категории {CATEGORY_TITLE}.")

Создано 355 заголовков статей в категории Category:Massachusetts Institute of Technology.


In [40]:
# Задаем секции, которые будут отброшены при парсинге статей
SECTIONS_TO_IGNORE = [
    'Robert Meservey', 'T. Colin Campbell',
    'Massachusetts Institute of Technology School of Engineering',
    'Sparcle', 'Deborah Leckband', 'RoboTuna', 'Busting Vegas',
]

In [41]:
# Функция возвращает список всех вложенных секций для заданной секции страницы Википедии
def all_subsections_from_section(
    section: mwparserfromhell.wikicode.Wikicode, # текущая секция
    parent_titles: list[str], # Заголовки родителя
    sections_to_ignore: set[str], # Секции, которые необходимо проигнорировать
) -> list[tuple[list[str], str]]:
    """
    Из раздела Википедии возвращает список всех вложенных секций.
    Каждый подраздел представляет собой кортеж, где:
      - первый элемент представляет собой список родительских секций, начиная с заголовка страницы
      - второй элемент представляет собой текст секции
    """

    # Извлекаем заголовки текущей секции
    headings = [str(h) for h in section.filter_headings()]
    title = headings[0]
    # Заголовки Википедии имеют вид: "== Heading =="

    if title.strip("=" + " ") in sections_to_ignore:
        # Если заголовок секции в списке для игнора, то пропускаем его
        return []

    # Объединим заголовки и подзаголовки, чтобы сохранить контекст для chatGPT
    titles = parent_titles + [title]

    # Преобразуем wikicode секции в строку
    full_text = str(section)

    # Выделяем текст секции без заголовка
    section_text = full_text.split(title)[1]
    if len(headings) == 1:
        # Если один заголовок, то формируем результирующий список
        return [(titles, section_text)]
    else:
        first_subtitle = headings[1]
        section_text = section_text.split(first_subtitle)[0]
        # Формируем результирующий список из текста до первого подзаголовка
        results = [(titles, section_text)]
        for subsection in section.get_sections(levels=[len(titles) + 1]):
            results.extend(
                # Вызываем функцию получения вложенных секций для заданной секции
                all_subsections_from_section(subsection, titles, sections_to_ignore)
                )  # Объединяем результирующие списки данной функции и вызываемой
        return results

# Функция возвращает список всех секций страницы, за исключением тех, которые отбрасываем
def all_subsections_from_title(
    title: str, # Заголовок статьи Википедии, которую парсим
    sections_to_ignore: set[str] = SECTIONS_TO_IGNORE, # Секции, которые игнорируем
    site_name: str = WIKI_SITE, # Ссылка на сайт википедии
) -> list[tuple[list[str], str]]:
    """
    Из заголовка страницы Википедии возвращает список всех вложенных секций.
    Каждый подраздел представляет собой кортеж, где:
      - первый элемент представляет собой список родительских секций, начиная с заголовка страницы
      - второй элемент представляет собой текст секции
    """

    # Инициализация объекта MediaWiki
    # WIKI_SITE ссылается на англоязычную часть Википедии
    site = mwclient.Site(site_name)

    # Запрашиваем страницу по заголовку
    page = site.pages[title]

    # Получаем текстовое представление страницы
    text = page.text()

    # Удобный парсер для MediaWiki
    parsed_text = mwparserfromhell.parse(text)
    # Извлекаем заголовки
    headings = [str(h) for h in parsed_text.filter_headings()]
    if headings: # Если заголовки найдены
        # В качестве резюме берем текст до первого заголовка
        summary_text = str(parsed_text).split(headings[0])[0]
    else:
        # Если нет заголовков, то весь текст считаем резюме
        summary_text = str(parsed_text)
    results = [([title], summary_text)] # Добавляем резюме в результирующий список
    for subsection in parsed_text.get_sections(levels=[2]): # Извлекаем секции 2-го уровня
        results.extend(
            # Вызываем функцию получения вложенных секций для заданной секции
            all_subsections_from_section(subsection, [title], sections_to_ignore)
        ) # Объединяем результирующие списки данной функции и вызываемой
    return results

In [42]:
# Разбивка статей на секции
wikipedia_sections = []
for title in titles:
    wikipedia_sections.extend(all_subsections_from_title(title))
print(f"Найдено {len(wikipedia_sections)} секций на {len(titles)} страницах")

Найдено 3010 секций на 355 страницах


In [43]:
# Очистка текста секции от ссылок <ref>xyz</ref>, начальных и конечных пробелов
def clean_section(section: tuple[list[str], str]) -> tuple[list[str], str]:
    titles, text = section
    # Удаляем ссылки
    text = re.sub(r"<ref.*?</ref>", "", text)
    # Удаляем пробелы вначале и конце
    text = text.strip()
    return (titles, text)

# Применим функцию очистки ко всем секциям с помощью генератора списков
wikipedia_sections = [clean_section(ws) for ws in wikipedia_sections]

# Отфильтруем короткие и пустые секции
def keep_section(section: tuple[list[str], str]) -> bool:
    """Возвращает значение True, если раздел должен быть сохранен, в противном случае значение False."""
    titles, text = section
    # Фильтруем по произвольной длине, можно выбрать и другое значение
    if len(text) < 16:
        return False
    else:
        return True

original_num_sections = len(wikipedia_sections)
wikipedia_sections = [ws for ws in wikipedia_sections if keep_section(ws)]
print(f"Отфильтровано {original_num_sections-len(wikipedia_sections)} секций, осталось {len(wikipedia_sections)} секций.")

Отфильтровано 268 секций, осталось 2742 секций.


In [44]:
# выведем 3 секции
for ws in wikipedia_sections[:3]:
    print(ws[0])
    display(ws[1][:50] + "...")
    print()

['Stata Center']


'{{Short description|Academic building in Cambridge...'


['Stata Center', '==Description==']


'[[File:View out the window of the Stata Center (20...'


['Stata Center', '==History==']


'{{Main|Building 20}}\nThe Stata Center is located o...'

In [45]:
GPT_MODEL = "gpt-3.5-turbo"  # only matters insofar as it selects which tokenizer to use

# Функция подсчета токенов
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Возвращает число токенов в строке."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Функция разделения строк
def halved_by_delimiter(string: str, delimiter: str = "\n") -> list[str, str]:
    """Разделяет строку надвое с помощью разделителя (delimiter), пытаясь сбалансировать токены с каждой стороны."""

    # Делим строку на части по разделителю, по умолчанию \n - перенос строки
    chunks = string.split(delimiter)
    if len(chunks) == 1:
        return [string, ""]  # разделитель не найден
    elif len(chunks) == 2:
        return chunks  # нет необходимости искать промежуточную точку
    else:
        # Считаем токены
        total_tokens = num_tokens(string)
        halfway = total_tokens // 2
        # Предварительное разделение по середине числа токенов
        best_diff = halfway
        # В цикле ищем какой из разделителей, будет ближе всего к best_diff
        for i, chunk in enumerate(chunks):
            left = delimiter.join(chunks[: i + 1])
            left_tokens = num_tokens(left)
            diff = abs(halfway - left_tokens)
            if diff >= best_diff:
                break
            else:
                best_diff = diff
        left = delimiter.join(chunks[:i])
        right = delimiter.join(chunks[i:])
        # Возвращаем левую и правую часть оптимально разделенной строки
        return [left, right]

# Функция обрезает строку до максимально разрешенного числа токенов
def truncated_string(
    string: str, # строка
    model: str, # модель
    max_tokens: int, # максимальное число разрешенных токенов
    print_warning: bool = True, # флаг вывода предупреждения
) -> str:
    """Обрезка строки до максимально разрешенного числа токенов."""
    encoding = tiktoken.encoding_for_model(model)
    encoded_string = encoding.encode(string)
    # Обрезаем строку и декодируем обратно
    truncated_string = encoding.decode(encoded_string[:max_tokens])
    if print_warning and len(encoded_string) > max_tokens:
        print(f"Предупреждение: Строка обрезана с {len(encoded_string)} токенов до {max_tokens} токенов.")
    # Усеченная строка
    return truncated_string

# Функция делит секции статьи на части по максимальному числу токенов
def split_strings_from_subsection(
    subsection: tuple[list[str], str], # секции
    max_tokens: int = 1000, # максимальное число токенов
    model: str = GPT_MODEL, # модель
    max_recursion: int = 5, # максимальное число рекурсий
) -> list[str]:
    """
    Разделяет секции на список из частей секций, в каждой части не более max_tokens.
    Каждая часть представляет собой кортеж родительских заголовков [H1, H2, ...] и текста (str).
    """
    titles, text = subsection
    string = "\n\n".join(titles + [text])
    num_tokens_in_string = num_tokens(string)
    # Если длина соответствует допустимой, то вернет строку
    if num_tokens_in_string <= max_tokens:
        return [string]
    # если в результате рекурсия не удалось разделить строку, то просто усечем ее по числу токенов
    elif max_recursion == 0:
        return [truncated_string(string, model=model, max_tokens=max_tokens)]
    # иначе разделим пополам и выполним рекурсию
    else:
        titles, text = subsection
        for delimiter in ["\n\n", "\n", ". "]: # Пробуем использовать разделители от большего к меньшему (разрыв, абзац, точка)
            left, right = halved_by_delimiter(text, delimiter=delimiter)
            if left == "" or right == "":
                # если какая-либо половина пуста, повторяем попытку с более простым разделителем
                continue
            else:
                # применим рекурсию на каждой половине
                results = []
                for half in [left, right]:
                    half_subsection = (titles, half)
                    half_strings = split_strings_from_subsection(
                        half_subsection,
                        max_tokens=max_tokens,
                        model=model,
                        max_recursion=max_recursion - 1, # уменьшаем максимальное число рекурсий
                    )
                    results.extend(half_strings)
                return results
    # иначе никакого разделения найдено не было, поэтому просто обрезаем строку (должно быть очень редко)
    return [truncated_string(string, model=model, max_tokens=max_tokens)]

In [46]:
# Делим секции на части
MAX_TOKENS = 1600
wikipedia_strings = []
for section in wikipedia_sections:
    wikipedia_strings.extend(split_strings_from_subsection(section, max_tokens=MAX_TOKENS))

print(f"{len(wikipedia_sections)} секций Википедии поделены на {len(wikipedia_strings)} строк.")

2742 секций Википедии поделены на 2773 строк.


In [47]:
# Напечатаем пример строки
print(wikipedia_strings[1])

Stata Center

==Description==

[[File:View out the window of the Stata Center (20 July 2004).jpg|thumb|upright=1.1|View from an upper-floor window]]
[[File:MIT Stata Center self-reflection.jpg|thumb|upright=1.1|Stata Center self reflection]]
In contrast to the MIT custom of referring to buildings by their numbers rather than their official names, the complex is usually referred to as "Stata" or "the Stata Center" (though the building number is still essential in identifying rooms at MIT). Above the fourth floor, the building splits into two distinct structures: the Gates Tower and the Dreyfoos Tower, often called "G Tower" and "D Tower" respectively.

The building has a number of small auditoriums and classrooms used by the Electrical Engineering and Computer Science department (EECS, Course 6), as well as other departments and on-campus groups.  Research labs and offices of the [[MIT Computer Science and Artificial Intelligence Laboratory|Computer Science and Artificial Intelligence L

In [48]:
EMBEDDING_MODEL = "text-embedding-ada-002"  # Модель токенизации от OpenAI
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")
client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))
# Функция отправки chatGPT строки для ее токенизации (вычисления эмбедингов)
def get_embedding(text, model="text-embedding-ada-002"):
   return client.embeddings.create(input = [text], model=model).data[0].embedding

Введите OpenAI API Key:··········


In [49]:
df = pd.DataFrame({"text": wikipedia_strings[:10]})
df['embedding'] = df.text.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
SAVE_PATH = "./MIT.csv"
# Сохранение результата
df.to_csv(SAVE_PATH, index=False)
df.tail(3)

,text,embedding
7,Stata Center\n\n== See also ==\n\n* [[List of ...,"[-0.012352237477898598, 0.01973605528473854, 0..."
8,Stata Center\n\n== References ==\n\n'''Notes''...,"[-0.012329358607530594, 0.0037225831765681505,..."
9,Stata Center\n\n== External links ==\n\n{{Comm...,"[-0.020836565643548965, 0.009624185971915722, ..."


In [50]:
# Функция поиска
def strings_ranked_by_relatedness(
    query: str, # пользовательский запрос
    df: pd.DataFrame, # DataFrame со столбцами text и embedding (база знаний)
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y), # функция схожести, косинусное расстояние
    top_n: int = 100 # выбор лучших n-результатов
) -> tuple[list[str], list[float]]: # Функция возвращает кортеж двух списков, первый содержит строки, второй - числа с плавающей запятой
    """Возвращает строки и схожести, отсортированные от большего к меньшему"""

    # Отправляем в OpenAI API пользовательский запрос для токенизации
    query_embedding_response = openai.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query,)

    # Получен токенизированный пользовательский запрос
    query_embedding = query_embedding_response.data[0].embedding

    # Сравниваем пользовательский запрос с каждой токенизированной строкой DataFrame
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()]

    # Сортируем по убыванию схожести полученный список
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)

    # Преобразовываем наш список в кортеж из списков
    strings, relatednesses = zip(*strings_and_relatednesses)

    # Возвращаем n лучших результатов
    return strings[:top_n], relatednesses[:top_n]

In [51]:
strings, relatednesses = strings_ranked_by_relatedness("curling gold medal", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.721


'Stata Center\n\n==Lawsuit==\n\n[[Image:MIT Stata Center-sprinker explosion-2007.jpg|thumb|upright=1.1|Water spray from a [[fire sprinkler system]] failure in 2007]]\nOn October 31, 2007,<ref>{{cite news|author=Dey, Arkajit|title=MIT Sues Gehry Firm Over Stata Problems; Lawsuit Describes Persistent Leaks, Sliding Ice and Snow, and Cracking Masonry|work=The Tech|url=http://tech.mit.edu/V127/N53/lawsuit.html|page=1\n|date=November 9, 2007|access-date=2007-11-09}}</ref><!-- if someone wants to add them, there are some good quotes and info in this The Tech piecwe, that were not in the articles in the Boston Globe or NY Times --> MIT sued<ref>{{cite news|author=Glahn, Lisa F. of Mintz Levin|title=Massachusetts Institute of Technology v. Frank O. Gehry & Associates, Inc., n/k/a Gehry Partners, LLP and Beacon Skanska Construction Company, n/k/a Skanska USA Building, Inc.|work=The Tech|url=http://tech.mit.edu/V127/N53/lawsuit/stata_lawsuit.pdf\n|date=October 31, 2007|access-date=2007-11-09}}</

relatedness=0.719


'Stata Center\n\n== Architectural criticism ==\n\nRobert Campbell, architecture columnist for \'\'[[The Boston Globe]]\'\', wrote a glowing appraisal of the building on April 25, 2004. According to Campbell, "the Stata is always going to look unfinished. It also looks as if it\'s about to collapse. Columns tilt at scary angles. Walls teeter, swerve, and collide in random curves and angles. Materials change wherever you look: brick, mirror-surface steel, [[brushed metal|brushed]] aluminum, brightly colored paint, corrugated metal. Everything looks improvised, as if thrown up at the last moment. That\'s the point. The Stata\'s appearance is a metaphor for the freedom, daring, and creativity of the research that\'s supposed to occur inside it." Campbell stated that the cost overruns and delays in completion of the Stata Center are of no more importance than similar problems associated with the building of [[St Paul\'s Cathedral]]. The 2005 [[Kaplan, Inc.|Kaplan]]/\'\'[[Newsweek]]\'\' guid

relatedness=0.713


"Stata Center\n\n== References ==\n\n'''Notes'''\n{{Reflist|30em}}\n\n'''Bibliography'''\n* {{cite book |first=Nancy E.|last=Joyce|others=afterword by William J. Mitchell, photography by Richard Sobol |title=Building Stata: The Design and Construction of Frank O. Gehry's Stata Center at MIT|publisher=[[The MIT Press]] |location=Cambridge, Massachusetts|year=2004\n|isbn=978-0-262-60061-3}}\n* {{cite book | first=William J.| last=Mitchell| title=Imagining MIT: Designing a Campus for the Twenty-First Century | year=2007 | publisher=[[The MIT Press]]|location=Cambridge, Massachusetts | isbn=978-0-262-13479-8}}"

relatedness=0.712


'Stata Center\n\n==Gallery==\n\n<gallery widths="200px" heights="200px" center mode=packed>\nImage:MIT Strata Center.jpg|Stata Center\nImage:The Ray and Maria Stata Center galawebdesign.jpg|View from the 7th floor\nImage:Stata Center-20050310-2.jpg|Interior, ground floor, Gates tower\nImage:MIT-Building32-from-54-at-night.jpg|Building 32 at night\nImage:Bldg 20 time capsule.jpg|Building 20 time capsule, on display in the Stata Center\n</gallery>'

relatedness=0.707


'Stata Center\n\n==History==\n\n{{Main|Building 20}}\nThe Stata Center is located on the site of the former Building 20, demolished in 1998. Building 20 had been erected hastily during World War II as a temporary building to house the historic [[Radiation Laboratory at the Massachusetts Institute of Technology|Radiation Laboratory]]. Over the course of 55 years, its "temporary" nature allowed research groups to have more space, and to make more creative use of that space, than was possible in more respectable buildings. The building also provided permanent rooms for official Institute clubs and groups, including the [[Tech Model Railroad Club]] and the MIT Electronic Research Society (MITERS).\n\nProfessor [[Jerome Lettvin|Jerome Y. Lettvin]] once quipped, "You might regard it as the womb of the Institute. It is kind of messy, but by God it is procreative!"'

In [52]:
# Функция формирования запроса к chatGPT по пользовательскому вопросу и базе знаний
def query_message(
    query: str, # пользовательский запрос
    df: pd.DataFrame, # DataFrame со столбцами text и embedding (база знаний)
    model: str, # модель
    token_budget: int # ограничение на число отсылаемых токенов в модель
) -> str:
    """Возвращает сообщение для GPT с соответствующими исходными текстами, извлеченными из фрейма данных (базы знаний)."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df) # функция ранжирования базы знаний по пользовательскому запросу
    # Шаблон инструкции для chatGPT
    message = 'Use the articles about MIT below to answer the following question. If you couldn’t find the answer in the articles, write “Couldn’t find the answer.”'
    # Шаблон для вопроса
    question = f"\n\nQuestion: {query}"

    # Добавляем к сообщению для chatGPT релевантные строки из базы знаний, пока не выйдем за допустимое число токенов
    for string in strings:
        next_article = f'\n\nWikipedia article section:\n"""\n{string}\n"""'
        if (num_tokens(message + next_article + question, model=model) > token_budget):
            break
        else:
            message += next_article
    return message + question

def ask(
    query: str, # пользовательский запрос
    df: pd.DataFrame = df, # DataFrame со столбцами text и embedding (база знаний)
    model: str = GPT_MODEL, # модель
    token_budget: int = 4096 - 500, # ограничение на число отсылаемых токенов в модель
    print_message: bool = False, # нужно ли выводить сообщение перед отправкой
) -> str:
    """Отвечает на вопрос, используя GPT и базу знаний."""
    # Формируем сообщение к chatGPT (функция выше)
    message = query_message(query, df, model=model, token_budget=token_budget)
    # Если параметр True, то выводим сообщение
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You answer questions about MIT."},
        {"role": "user", "content": message},
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0 # гиперпараметр степени случайности при генерации текста. Влияет на то, как модель выбирает следующее слово в последовательности.
    )
    response_message = response.choices[0].message.content
    return response_message

In [61]:
# Настройки бота
logging.basicConfig(level=logging.ERROR)
API_TOKEN = <TELEGRAMM TOKEN>
bot = Bot(token=API_TOKEN)
dp = Dispatcher()

In [62]:
# Хэндлеры
@dp.message(Command("start"))
async def cmd_start(message: types.Message):

    name = message.from_user.first_name
    await message.answer("👋")
    time.sleep(2)
    await message.answer(f"Привет, {name} !")
    time.sleep(2)
    await message.answer("Я бот. У меня под 'под капотом' GPT-3.5 Turbo!")
    time.sleep(2)
    await message.answer("Для получения информации набери '/help' ")

@dp.message(Command("help"))
async def cmd_start(message: types.Message):
    await message.answer("Я отвечу на твои вопросы про 'Массачусетский технологический институт'")
    time.sleep(2)
    await message.answer("В моей базе знаний 2773 записей")
    time.sleep(2)
    await message.answer("Вот пример запроса к моей базе:")
    time.sleep(2)
    await message.answer("What disciplines are taught in MIT?")
    time.sleep(2)
    await message.answer("⬇️⬇️⬇️ Задай вопрос ⬇️⬇️⬇️")

@dp.message()
async def cmd_start(message: types.Message):
    txt = message.text
    await message.answer(ask(str(txt)))

In [64]:
# Поллинг
async def main():
    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())